In [38]:
import numpy as np
import csv

import os

import matplotlib.pyplot as plt
from copy import deepcopy

#import collections

train_bool=True

In [39]:

def obtain_data(train_bool=True):
    if train_bool:
        volume_filename = "volume_train.csv"
    else:
        volume_filename = "volume_test.csv"

    with open(volume_filename) as csvfile:
        rows = [row for row in csv.DictReader(csvfile)]

    fields = set([k for (k,_) in rows[0].items()])
    print("Fields:", fields)

    with open("raw_2010.csv") as csvfile:
        raw_rows = [row for row in csv.DictReader(csvfile)]
    with open("raw_2011.csv") as csvfile:
        raw_rows += [row for row in csv.DictReader(csvfile)]
    
    return rows, raw_rows
    


In [40]:
# pvalues = {}
# key: station #
# value: p value
def obtain_pvalues(exp_num=2, train_bool=True):
    pv_fnames = ["ExtremeTempTraffic","TempTraffic","WeatherTraffic","WeekendWeekday"]
    fname = pv_fnames[exp_num]

    if train_bool:
        fname += "_Train"
    else:
        fname += "_Test"
    
    filename = os.path.join("pvalues", fname +".csv")
    with open(filename) as csvfile:
        pvalues = {float(row["stations"]): float(row["station.p.vals"]) for row in csv.DictReader(csvfile)}
    
    return pvalues



In [41]:
rows, raw_rows = obtain_data(train_bool=train_bool)


Fields: {'', 'daily.data.apparentTemperatureHigh', 'daily.data.precipType', 'daily.data.temperatureLow', 'Date', 'Month', 'daily.data.windSpeed', 'Season', 'Month_y', 'daily.data.icon', 'Month_x', 'daily.data.apparentTemperatureLow', 'daily.data.precipIntensity', 'Outflow', 'Total', 'Inflow', 'daily.data.temperatureHigh', 'X', 'Precip_bin', 'Wday_bin', 'Wday', 'daily.data.humidity', 'daily.data.precipProbability', 'daily.data.summary', 'Hday_bin', 'Station'}


In [42]:
# For myself and my project partners

# SDR: Station Date Rides

# key: (station, date)
# value: list of indices of raw data 
def obtain_sdr(raw_rows):
    station_date_rides = {}
    for i,row in enumerate(raw_rows):
        date = row["Start date"].split(" ")[0]

        station = row["Start station number"]
        if not (station, date) in station_date_rides.keys():
            station_date_rides[(station, date)] = []
        station_date_rides[(station, date)].append(i)

        station = row["End station number"]
        if not (station, date) in station_date_rides.keys():
            station_date_rides[(station, date)] = []
        station_date_rides[(station, date)].append(i)
    return station_date_rides

# Output to CSV with 1-indexing, for correct use in R
def output_sdr_csv():
    station_date_rides = obtain_sdr()
    
    # use i+1 so that indexing is as in R
    for k, ilist in station_date_rides.items():
        ilist = [i+1 for i in ilist]

    with open("station_date_trips.csv", "w+") as outfile:
        #wrt = csv.writer(outfile)# quoting=csv.QUOTE_NONE)
        wrt = csv.writer(outfile, quoting=csv.QUOTE_ALL)
        wrt.writerow(["Station", "Date", "Trips"])
        for k,val in station_date_rides.items():
            station, date = k
            third = " ".join(tuple(str(v) for v in val))
            #wrt.writerow([station, date, '"' + third + '"'])
            wrt.writerow([station, date,third])
    

In [43]:

# key: station
# value: dict of daily traffic values with: 
#   (key: date, val: traffic level value)
def obtain_station_days_traffic(rows):
    station_days = {}
    for row in rows:
        station = row["Station"]
        if not station in station_days.keys():
            station_days[station] = {}
        date = row["Date"]
        station_days[station][date] = row["X"]
    return station_days
    
# Return indices of those discoveries made
def bh(pv_dict, alpha=.05, storey=False, printing=False):
    if storey:
        alpha /= storey_pihat(pvs)
        
    n = float(len(pv_dict))
    k = int(n) -1
    pvs = [(p,i) for i,p in enumerate(pv_dict)]
    pvs.sort()
    while not pvs[k-1][0] <= (alpha*k)/n:
        if printing:
            print(pvs[k-1][0], (alpha*k)/n)
        k -= 1
        if k == 0:
            break
    return [pv[1] for pv in pvs[:k]]

#def storey_bh(pvs, alpha, gamma=0.5):
def storey_pihat(pv_dict, gamma=0.5):
    num = 0.
    for _, pv in pvs.items():
        if pv > gamma:
            num += 1.
    denom = len(pvs) * (1.-gamma)
    #if denom:
    pihat = min(num/denom, 1)
    return pihat
    
#  function
def group_adaptive_bh(pv_dict, groups_indices, alpha=.05, gamma=0.5):    
    groups = [[p_vector[i] for i in gi] for gi in groups_indices]
    # Pi hat for each group
    group_pi_hats = [storey_pihat(group) for group in groups]

    for i,group in enumerate(groups_indices):
        for j in group:
            p_vector[j] *= group_pi_hats[i]
            
    for i,p in enumerate(p_vector):
        if p > gamma:
            p_vector[i] = np.infty

    # Now do ordinary BH on mod_p_vector
    return bh(p_vector, alpha)

def old_group_adaptive_bh(p_vector, groups_indices, alpha=.05, gamma=0.5):    
    groups = [[p_vector[i] for i in gi] for gi in groups_indices]
    # Pi hat for each group
    group_pi_hats = [storey_pihat(group) for group in groups]

    for i,group in enumerate(groups_indices):
        for j in group:
            p_vector[j] *= group_pi_hats[i]
            
    for i,p in enumerate(p_vector):
        if p > gamma:
            p_vector[i] = np.infty

    # Now do ordinary BH on mod_p_vector
    return bh(p_vector, alpha)
    
    
    

In [53]:
# station_days contains train vs test information
# tells us which subset of raw_rows to use
#def obtain_raw_subset(raw_rows, station_days)


def obtain_duration_groups(rows, raw_rows, station_date_rides):
    # key: station
    # value: (total duration, number of trips)
    # then can compute avg by taking v[0]/v[1]
    station_durations = {}
    for row in rows:
        station = row["Station"]
        date = row["Date"]    
        if (station, date) in station_date_rides:
            
            if not station in station_durations.keys():
                station_durations[station] = [0,0]

            total_duration = 0
            
            for i in station_date_rides[(station, date)]:
                total_duration += float(raw_rows[i]["Duration"])

            station_durations[station][0] += total_duration
            station_durations[station][1] += len(station_date_rides[(station, date)])
    
    sort_station_durations = [(v[0]/v[1],k) for k,v in station_durations.items()]
    sort_station_durations.sort()
    #sort_station_durations_names = sort_station_durations
    sort_station_durations_names = [k for (_,k) in sort_station_durations]
    
    nstations = len(sort_station_durations_names)
    ngroups = 4
    ratio = int(nstations/ngroups)
    
    groups = []
    for i in range(ngroups):
        groups.append(sort_station_durations_names[ratio*i:ratio*(i+1)])
    # fill the last group with stragglers, if rounding error on indices
    groups[-1] += sort_station_durations_names[ratio*ngroups:]
    return groups

def obtain_membership_groups(rows, raw_rows, station_date_rides):
    # key: station
    # value: (number of member trips, total number of trips)
    # then can compute rate of membership by taking v[0]/v[1]
    station_rates = {}
    for row in rows:
        station = row["Station"]
        date = row["Date"]    
        if (station, date) in station_date_rides:
            
            if not station in station_rates.keys():
                station_rates[station] = [0,0]

        
            member_rides = 0
            
            for i in station_date_rides[(station, date)]:
                member_rides += (raw_rows[i]["Member type"] == "Member")

            station_rates[station][0] += member_rides
            station_rates[station][1] += len(station_date_rides[(station, date)])
        
    
    sort_station_durations = [(v[0]/v[1],k) for k,v in station_rates.items()]
    sort_station_durations.sort()
    #sort_station_durations_names = sort_station_durations
    sort_station_durations_names = [k for (_,k) in sort_station_durations]
    
    nstations = len(sort_station_durations_names)
    ngroups = 4
    ratio = int(nstations/ngroups)
    
    groups = []
    for i in range(ngroups):
        groups.append(sort_station_durations_names[ratio*i:ratio*(i+1)])
    # fill the last group with stragglers, if rounding error on indices
    groups[-1] += sort_station_durations_names[ratio*ngroups:]
    return groups

    

In [45]:
station_date_rides = obtain_sdr(raw_rows)
#groups = obtain_duration_groups(rows, raw_rows, station_date_rides)
#groups = obtain_membership_groups(rows, raw_rows, station_date_rides)


#for group in groups:
    #print((group))


In [54]:

# (station, pvalue) tuples
def index_groups(groups, pvalues):
    groups = [[int(g) for g in group] for group in groups]
    return groups
    igroups = []
    for group in groups:
        igroup = []
        for i in range(len(pvalues)):
            if pvalues[i][0] in group:
                igroup.append(i)
        igroups.append(igroup)
    return igroups

def execute_all(rows, raw_rows, station_date_rides, train_bool=True):
    alpha=.05
    
    duration_groups = obtain_duration_groups(rows, raw_rows, station_date_rides)
    
    membership_groups = obtain_membership_groups(rows, raw_rows, station_date_rides)

    results = []
    pvaluess = []
    for exp_num in range(4):
        pvalues = list(obtain_pvalues(exp_num=exp_num, train_bool=train_bool).items())
        pvaluess.append(pvalues)
        #plist = [p for (_,p) in pvalues.items()]
        plist = [p for (_,p) in pvalues]
        
        duration_igroups = index_groups(duration_groups, pvalues)
        membership_igroups = index_groups(membership_groups, pvalues)

        result = []
        result.append(bh(plist, alpha=alpha, storey=False))
        result.append(bh(plist, alpha=alpha, storey=True))
        result.append(group_adaptive_bh(plist, duration_igroups, alpha=alpha))
        result.append(group_adaptive_bh(plist, membership_igroups, alpha=alpha))
        results.append(result)
        
    return results, pvaluess
        
            


In [55]:
results, pvaluess = execute_all(rows, raw_rows, station_date_rides, train_bool=train_bool)

In [57]:
def analyze_results(results):
    for result in results:
        rlens = [len(r) for r in result]
        minn = min(rlens)
        maxx = max(rlens)
        print("min, max:", minn, maxx)
        print(rlens)

analyze_results(results)
print(results)



#print(min([4,5,2,3]))

if False:
    #station_days_traffic = obtain_station_days_traffic(rows)
    print(station_date_rides[('31200', '2011-02-07')])

    for row in raw_rows:
        if row["Start station number"] == "31404": #and row["Date"][:10] == '2011-02-07':
            print(row)    
        if row["End station number"] == "31404": #and row["Date"][:10] == '2011-02-07':
            print(row)

min, max: 0 64
[0, 2, 2, 64]
min, max: 106 119
[106, 109, 112, 119]
min, max: 0 1
[0, 0, 0, 1]
min, max: 14 14
[14, 14, 14, 14]
[[[], [26, 113], [113, 26], [113, 26, 0, 37, 95, 13, 51, 76, 88, 44, 103, 99, 97, 46, 56, 91, 11, 61, 136, 25, 45, 101, 104, 72, 65, 71, 75, 66, 38, 82, 139, 40, 12, 60, 55, 80, 90, 81, 33, 86, 17, 68, 87, 58, 70, 79, 14, 102, 93, 30, 35, 52, 1, 92, 24, 77, 83, 57, 32, 100, 122, 41, 53, 43]], [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 27, 28, 30, 31, 32, 33, 34, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 105, 29, 41, 50, 54, 94, 103, 42, 49, 75, 24, 64, 22, 79, 36, 113, 104, 26, 35, 114], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 27, 28, 30, 31, 32, 33, 34, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48,

In [25]:
if False:
    for row in rows:
        if row["Station"] == "31404": #and row["Date"][:10] == '2011-02-07':
            print(row)    

    #station_dates = [(station,day)  for (station,dct) in station_days_traffic.items() for day in dct.keys()]
    #print( ( station_dates ))


In [90]:


def parse_loc_data(stations, rejections):
    
    with open("location_data.csv") as csvfile:
        rows = [row for row in csv.DictReader(csvfile)]
    
    results = []
    for row in rows:
        rejection = rejections[row["Station"]]
        result = [row["Latitude"], row["Longitude"], str(rejection)] 
    
    with open("loc_coords.csv", "w+") as outfile:
        wrt = csv.writer(outfile, quoting=csv.QUOTE_NONE)
        wrt.writerow(["Lat", "Lon", "Rejected"])
        for res in result:
            wrt.writerow(res)
#parse_loc_data()